# CS486 - Artificial Intelligence
## Lesson 11 - Minimax

In adversarial search, our agent only gets to choose some portion of the path to the goal. How does that impact our search? We'll walk through a few examples of finding the best path to goal in the presence of an adversary.  

In [ ]:
from helpers import nim_decision_tree
from aima.games import *
from aima.notebook import psource

## Nim

Nim is a game in which two players are given a pile of objects and take turns taking 1 to $n$ objects from the pile. The player that takes the last object loses.  

To make it a little more interesting, let's say that the winner's score is the number of objects that their ** *opponent* ** took. In other words, you want to win the game having taken the fewest number of objects possible. We'll call our variation of the game *Mini*nim. Here's an implementation:

In [ ]:
class Mininim(Game):
    def __init__(self,objects,moves):
        self.moves = list(range(1,moves+1))
        self.initial = GameState(to_move=1,utility=0,board=objects,moves=self.moves)
    
    def actions(self,state):
        return [x for x in self.moves if x <= state.board]
    
    def result(self,state,action):
        to_move=state.to_move%2+1
        
        # we save the number of objects player 1 has 
        # taken as the utility in the game state.
        if state.to_move == 1:
            utility=state.utility+action
        else:
            utility=state.utility
        
        board=state.board-action
        moves=[x for x in self.moves if x <= board]
        
        return GameState(to_move,utility,board,moves)
    
    def utility(self,state,player):
        if state.to_move == 1:
            score = self.initial.board - state.utility
        else:
            score = -state.utility
            
        return (score if player == 1 else -score)

## Adversarial Search

Consider the following decision tree for a Mininim game with three objects and two players that alternate in taking one or two objects from the pile at a time. Which path yields the highest score? What is the best opening move for the first player? Why aren't they the same?

In [ ]:
nim_decision_tree(objects=3,moves=2)

So let's play the 3-pile game from the decision tree above with two random players. The output of the play is **player 1's** score. It's negative if player 1 loses. Let's see a few games against random players:

In [ ]:
mininim.play_game(query_player,random_player)

## Playing Nim

There are three kinds of player that can play AIMA games:

* `random_player`: Plays a random action
* `minimax_player`: Defined below. Does an exhaustive search of the minimax tree and moves accordingly. 
* `query_player`: Queries you for a play.
* `alphabeta_player`: Minimax player with alpha beta pruning. 

We'll play a 5-pile game with each kind of player. The output of the play is player 1's score. It's negative if player 1 loses. First, a random player:

In [ ]:
mininim = Mininim(objects=5,moves=2)
mininim.play_game(query_player,random_player)

In [ ]:
minimax_player = lambda game,state: minimax_decision(state,game)
mininim.play_game(query_player,minimax_player)

In [ ]:
minimax_player = lambda game,state: minimax_decision(state,game)
mininim.play_game(minimax_player,query_player)

## Alpha-Beta Pruning 

So what's the difference between the minimax player and the alphabeta pruning player? Run the blocks below that time how long it takes the player to decide their first move:

In [ ]:
%%time
large_pile_mininim = Mininim(objects=30,moves=3)
move = alphabeta_search(mininim.initial,large_pile_mininim)
print(move)

In [ ]:
%%time
large_pile_mininim = Mininim(objects=30,moves=3)
move = minimax_decision(large_pile_mininim.initial,mininim)
print(move)

What's the difference? Pruning plays a huge role in how deep an agent can search. Working from the left to right, label the value of each node in the decision tree below. How far do you get before you can start to prune?

In [ ]:
nim_decision_tree(objects=5,moves=2)

A fun variation on Nim is [Race to 50](https://mindyourdecisions.com/blog/2013/03/12/dice-game-race-to-50-video/). Just make sure you go first :)